In [443]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

In [444]:
# data = input_data.read_data_sets('input/data')#, source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/')
data = pd.read_csv('spambase.data.txt', header=None)
data.rename(columns={57:'is_spam'}, inplace=True)
X_train = []
Y_train = []
with open("spambase.data.txt") as f:
    for i,line in enumerate(f):
        line= line.strip()
        line = line.split(',')
        line2 = [float(i) for i in line]
        X_train.append(line2[:len(line2)-2])
        Y_train.append(line2[len(line2)-1:][0])
session = tf.InteractiveSession()

In [445]:
X_train = np.array(X_train)

In [447]:
# Training Parameters
learning_rate = 0.000001
num_steps = 4100
batch_size = 1

display_step = 100


# Network Parameters
num_hidden_1 = 20 # 1st layer num features
num_input = 56 # MNIST data input (img shape: 28*28)

# tf Graph input (only pictures)
X = tf.placeholder("float32", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([num_input])),
}

In [448]:

def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    return layer_1


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    return layer_1

In [449]:
# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [450]:
sess = tf.Session()
sess.run(init)


# Training
for i in range(1):
    for index in range(num_steps):
        # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
        batch_x = X_train[index * batch_size: (index + 1) * batch_size]
    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
    # Display logs per step
        if index % display_step == 0 or index == 1:
            print('Step %i: Minibatch Loss: %f' % (index, l))


    

Step 0: Minibatch Loss: 66.429367
Step 1: Minibatch Loss: 182.261795
Step 100: Minibatch Loss: 1.090351
Step 200: Minibatch Loss: 350.097168
Step 300: Minibatch Loss: 64.002380
Step 400: Minibatch Loss: 0.580922
Step 500: Minibatch Loss: 4.756402
Step 600: Minibatch Loss: 664.643555
Step 700: Minibatch Loss: 55.914375
Step 800: Minibatch Loss: 3564.736572
Step 900: Minibatch Loss: 11.255156
Step 1000: Minibatch Loss: 9016.593750
Step 1100: Minibatch Loss: 26.295574
Step 1200: Minibatch Loss: 369.945526
Step 1300: Minibatch Loss: 6.237895
Step 1400: Minibatch Loss: 3.353933
Step 1500: Minibatch Loss: 85.486534
Step 1600: Minibatch Loss: 288.014709
Step 1700: Minibatch Loss: 0.321673
Step 1800: Minibatch Loss: 2.745939
Step 1900: Minibatch Loss: 1.379956
Step 2000: Minibatch Loss: 2.981306
Step 2100: Minibatch Loss: 4.764340
Step 2200: Minibatch Loss: 2.574328
Step 2300: Minibatch Loss: 0.380571
Step 2400: Minibatch Loss: 2.670952
Step 2500: Minibatch Loss: 0.720108
Step 2600: Minibatch 

In [451]:
reconstructed_data = []
g = sess.run(decoder_op, feed_dict={X:X_train})
reconstructed_data = list(g)

In [452]:
np.array(reconstructed_data).shape

(4601, 56)

In [453]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
log_reg = linear_model.LogisticRegression()
log_reg.fit(reconstructed_data[:4000], Y_train[:4000])
predicted_labels = log_reg.predict(reconstructed_data[4000:])
score = accuracy_score(Y_train[4000:],predicted_labels)
print("Accuracy of predictions: " ,score * 100, "%")

Accuracy of predictions:  74.20965058236273 %


In [454]:
reconstructed_data_reduced_dim = []
g = sess.run(encoder_op, feed_dict={X:X_train})
reconstructed_data_reduced_dim = list(g)

In [455]:
np.array(reconstructed_data_reduced_dim).shape

(4601, 20)

In [456]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score
log_reg = linear_model.LogisticRegression()
log_reg.fit(reconstructed_data_reduced_dim[:4000], Y_train[:4000])
predicted_labels = log_reg.predict(reconstructed_data_reduced_dim[4000:])
score = accuracy_score(Y_train[4000:],predicted_labels)
print("Accuracy of predictions: " ,score * 100, "%")

Accuracy of predictions:  72.04658901830283 %
